In [80]:
import requests
import pandas as pd 
import json
import concurrent.futures
import re
import queue
import os

from collections import defaultdict
import unicodedata

In [2]:
df = pd.read_csv("DATA.csv")
df.head()

,Thể loại,Nguồn nhập
0,VĂN HỌC,NaN
1,Văn học đương đại,https://nhasachphuongnam.com/vi/van-hoc-duong-...
2,Văn học kinh điển – cổ điển,https://www.fahasa.com/sach-trong-nuoc/van-hoc...
3,Văn học kinh điển – cổ điển,https://nhasachphuongnam.com/vi/van-hoc-kinh-d...
4,Văn học kinh điển – cổ điển,https://tiki.vn/tac-pham-kinh-dien/c842


In [3]:
df2 = pd.read_csv("DATA_v2.csv")
df2.dropna(inplace = True) 
df2.columns = ["Thể loại", "Nguồn nhập", "type"]

# Remove all row that not have tiki
df2 = df2[df2['Nguồn nhập'].str.contains("tiki")]
group_f1 = df2.groupby("Thể loại")["Nguồn nhập"].apply(list).to_dict()
group_f2 = df2.groupby("type")["Thể loại"].apply(list).to_dict()

n_dict = defaultdict(dict)
for k, v in group_f2.items():
    for stype in v:
        n_dict[k][stype] = group_f1[stype]

In [11]:
import json
with open("category.json", "w",encoding='utf-8') as outfile:
    json.dump(n_dict,outfile,indent = 4,ensure_ascii=False)

In [4]:
_category_path = "https://tiki.vn/api/personalish/v1/blocks/listings?limit=100&include=advertisement&aggregations=1&category={}&page={}&urlKey={}"
_book_detail = 'https://tiki.vn/api/v2/products/{}?platform=web&spid={}'

In [60]:
url = "https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&include=advertisement&aggregations=1&trackity_id=c0a5b178-9ff2-3eaa-d2c2-ce87bba5db01&categoryId=316"
_book_detail = 'https://tiki.vn/api/v2/products/{}?platform=web&spid=119384919'
payload={}
headers = {
  'authority': 'tiki.vn',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
  'accept': 'application/json, text/plain, */*',
  'sec-ch-ua-mobile': '?0',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty',
  'referer': 'https://tiki.vn/sach-truyen-tieng-viet/c316',
  'accept-language': 'vi,en-US;q=0.9,en;q=0.8,vi-VN;q=0.7'
}

In [6]:
_category = {}
_reverse_mapping = {}
for _ in response['filters'][0]['values']:
    _category[_['url_key']] = _['query_value']
    
for _ in response['filters'][0]['values']:
    _reverse_mapping[_['query_value']] = _['display_value']

In [8]:
 _res = requests.request("GET", _category_path.format('4933', 0, 'du-ky'), headers=headers, data=payload).json()

In [18]:
print(_res['data'][0]['id'])
print(_res['data'][0]['seller_product_id'])

77160746
77160747


In [20]:
__res =  requests.request("GET",_book_detail.format(77160746,77160747), headers=headers)

In [21]:
__res.json()

{'id': 77160746,
 'master_id': 77160746,
 'sku': '1568397233771',
 'name': 'Người tình Havana',
 'url_key': 'nguoi-tinh-havana-p77160746',
 'url_path': 'nguoi-tinh-havana-p77160746.html?spid=77160747',
 'type': 'simple',
 'book_cover': None,
 'short_description': 'Tình yêu quan trọng nhất và cũng khó học nhất trên cõi đời này là gì? Là tình yêu dành cho người bạn đời, tình yêu cuộc sống hay tình yêu dành cho chính bản thân mình?\xa0\xa0Cô gái dữ dội và mạnh mẽ của ...',
 'price': 90000,
 'list_price': 90000,
 'original_price': 90000,
 'badges': [{'code': 'new_pdp', 'text': 'v1'}],
 'badges_new': [{'placement': 'pre_bottom',
   'code': 'best_price_guaranteed',
   'type': 'minor_badge',
   'icon': 'https://salt.tikicdn.com/ts/upload/2e/da/c9/4b9c0150392c753ccb65b2595500e9d6.png',
   'icon_width': 130,
   'icon_height': 24,
   'text_color': None,
   'background_color': None,
   'href': 'https://hotro.tiki.vn/s/topic/0TO5Y00000BiIPaWAN/ch%C6%B0%C6%A1ng-tr%C3%ACnh-r%E1%BA%BB-h%C6%A1n-ho%C3%

_exmaple = {Tên sách, Ảnh bìa, Tên Tác giả, Nhà Xuất bản, Thể loại, tóm tắt, giá

In [88]:
_big_dict = {}
# booksQueue = queue.Queue()

class Crawler():
    def __init__(self, categoryid, urlkey, cname):
        self.categoryid = categoryid
        self.urlkey = urlkey
        self.category_name = cname
        
    def crawl(self):
        _concurrent = self._bookshelf()
        print(_concurrent)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()*5) as executor:
            executor.map(self._books, _concurrent)
            
    def _books(self, book_tuple):
        
        book = {}
        
        __res =  requests.request("GET",_book_detail.format(book_tuple[0],book_tuple[1]), headers=headers).json()
        book['Tên Sách'] = __res['name']
        book['Ảnh bìa'] = __res['thumbnail_url']
        for _spec in __res['specifications'][0]['attributes']:
            book[_spec['name']] = _spec['value']
            
            _big_dict[_spec['name']] = ''
#         book['Nhà phát hành'] = __res['specifications'][0]['attributes'][0]["value"]
#         book['Nhà xuất bản'] = __res['specifications'][0]['attributes'][-1]["value"]
#         book['Dịch Giả'] = __res['specifications'][0]['attributes'][3]["value"]
#         book['Số trang'] = __res['specifications'][0]['attributes'][3]["value"]
#         book['Ngày xuất bản'] = __res['specifications'][0]['attributes'][1]["value"]
        
        book['Tác giả'] = ','.join([author['name'] for author in __res['authors']])
        book['Thể loại'] = self.category_name
        book['Giá bìa'] = __res['price']
#         book['Nội dung tóm tắt'] = __res['short_description']
        book['Nội dung tóm tắt'] = unicodedata.normalize("NFKD",__res['short_description'])
        
        booksQueue.put(book)
            
        print(f"Number of products in my pocket {booksQueue.qsize()}")
        
        
    def _bookshelf(self):
        
        _bookshelf = []
        _res = requests.request("GET", _category_path.format(self.categoryid, 0, self.urlkey), headers=headers).json()
        _total = _res['paging']['last_page']+1
        
        for _ in range(_total):
            print(f"total page of {self.urlkey} : {_total}")
            _res = requests.request("GET", _category_path.format(self.categoryid, _, self.urlkey), headers=headers).json()
            for data in _res['data']:
                _bookshelf.append( (data['id'],data['seller_product_id']) )
            break
        
        return _bookshelf

In [97]:
__res.json()['thumbnail_url']

'https://salt.tikicdn.com/cache/280x280/ts/product/77/e9/42/e37e4d64caec99c6f5462c31f4c637df.jpg'

In [16]:
_url = 'https://tiki.vn/du-ky/c4933'
_split = _url.split("/")
_split

['https:', '', 'tiki.vn', 'du-ky', 'c4933']

In [89]:
t = Crawler(_split[-1][1:],_split[-2],'Chăm sóc sức khỏe')
# print(t._bookshelf())
# print(t._books((77160746, 77160747)))
t.crawl()

total page of du-ky : 3
[(77160746, 77160747), (3577373, 3609471), (76106026, 76262768), (53545646, 53545647), (32314535, 34747302), (17406542, 17406543), (4085769, 4085771), (62349850, 62349851), (70722135, 70722136), (14100924, 14100925), (117595287, 119382968), (109982962, 109982963), (387725, 20491958), (10190801, 10190802), (14684956, 14684957), (576158, 87495949), (497538, 212412), (26412706, 26412707), (548212, 20491764), (14523892, 14523893), (23943907, 23943908), (10669577, 10669578), (22367423, 59005374), (113583666, 113583669), (105278809, 105278810), (113353587, 120115793), (554645, 110309450), (565422, 20470851), (15592528, 15592531), (644149, 24825210), (5995385, 5995387), (477659, 188098), (11491784, 11491785), (15867080, 15867081), (111582626, 111582628), (107059011, 107059012), (24593377, 118083655), (109535389, 109535390), (53720985, 54138841), (10738152, 20488984), (115482831, 115482832), (10707066, 56273931), (5745743, 20477853), (51550236, 55751890), (48481757, 484

In [91]:
booksQueue.get()

{'Tên Sách': 'Người tình Havana',
 'Ảnh bìa': 'https://salt.tikicdn.com/cache/280x280/ts/product/77/e9/42/e37e4d64caec99c6f5462c31f4c637df.jpg',
 'Công ty phát hành': 'NXB Hội Nhà Văn',
 'Ngày xuất bản': '2020-12-04 17:47:13',
 'Loại bìa': 'Bìa mềm',
 'Số trang': '260',
 'Nhà xuất bản': 'Nhà Xuất Bản Hội Nhà Văn',
 'Tác giả': 'Đinh Hằng',
 'Thể loại': 'Chăm sóc sức khỏe',
 'Giá bìa': 90000,
 'Nội dung tóm tắt': 'Tình yêu quan trọng nhất và cũng khó học nhất trên cõi đời này là gì? Là tình yêu dành cho người bạn đời, tình yêu cuộc sống hay tình yêu dành cho chính bản thân mình?  Cô gái dữ dội và mạnh mẽ của ...'}

In [100]:

for _name in n_dict.keys():
    booksQueue = queue.Queue()
    print(_name)
    for k,v in n_dict[_name].items():
        print(v)
        _split = v[0].split("/")
        print(_split)
        t = Crawler(_split[-1][1:],_split[-2], k)
        t.crawl()
        
        df = pd.DataFrame(columns=['Tên Sách', 'Ảnh bìa','Thể loại','Tác giả','Nội dung tóm tắt','Giá bìa']+ list(_big_dict.keys()),index=[0])
        while not booksQueue.empty():
            df = df.append(booksQueue.get(),ignore_index=True)
                           
        df.to_csv('tiki_'+ _name+'.csv')

CHĂM SÓC GIA ĐÌNH
['https://tiki.vn/cham-soc-suc-khoe/c863']
['https:', '', 'tiki.vn', 'cham-soc-suc-khoe', 'c863']
['https://tiki.vn/sach-lam-cha-me/c3320']
['https:', '', 'tiki.vn', 'sach-lam-cha-me', 'c3320']
['https://tiki.vn/sach-ky-nang-cho-be/c3271']
['https:', '', 'tiki.vn', 'sach-ky-nang-cho-be', 'c3271']
['https://tiki.vn/sach-dinh-duong-suc-khoe-cho-be/c3272']
['https:', '', 'tiki.vn', 'sach-dinh-duong-suc-khoe-cho-be', 'c3272']
['https://tiki.vn/sach-thai-giao/c6138']
['https:', '', 'tiki.vn', 'sach-thai-giao', 'c6138']
HỌC NGÔN NGỮ
['https://tiki.vn/sach-hoc-tieng-han/c1526']
['https:', '', 'tiki.vn', 'sach-hoc-tieng-han', 'c1526']
['https://tiki.vn/sach-hoc-tieng-nhat/c1524']
['https:', '', 'tiki.vn', 'sach-hoc-tieng-nhat', 'c1524']
NGÀNH CÔNG NGHỆ - THÔNG TIN
['https://tiki.vn/lap-trinh/c9723']
['https:', '', 'tiki.vn', 'lap-trinh', 'c9723']
['https://tiki.vn/tin-hoc-van-phong/c9721']
['https:', '', 'tiki.vn', 'tin-hoc-van-phong', 'c9721']
['https://tiki.vn/thiet-ke-do-h

In [94]:
df = pd.DataFrame(columns=['Tên Sách', 'Ảnh bìa','Thể loại','Tác giả','Nội dung tóm tắt','Giá bìa']+ list(_big_dict.keys()),index=[0])
while not booksQueue.empty():
    df = df.append(booksQueue.get(),ignore_index=True)
                           
df.to_csv('tiki_'+ 'Chăm sóc sức khỏe'+'.csv')

In [96]:
df['Ảnh bìa']

0                                                    NaN
1      https://salt.tikicdn.com/cache/280x280/ts/prod...
2      https://salt.tikicdn.com/cache/280x280/ts/prod...
3      https://salt.tikicdn.com/cache/280x280/ts/prod...
4      https://salt.tikicdn.com/cache/280x280/ts/prod...
                             ...                        
103    https://salt.tikicdn.com/cache/280x280/ts/prod...
104    https://salt.tikicdn.com/cache/280x280/ts/prod...
105    https://salt.tikicdn.com/cache/280x280/ts/prod...
106    https://salt.tikicdn.com/cache/280x280/ts/prod...
107    https://salt.tikicdn.com/cache/280x280/ts/prod...
Name: Ảnh bìa, Length: 108, dtype: object

In [83]:
for _url,_id in _category.items():
    print(_id,_url)
    t = Crawler(_id,_url)
    t.crawl()

839 sach-van-hoc
total pages 100
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
total books of 1 is  150
Number of products in my pocket 150
total books of 4 is  150
Number of products in my pocket 300
total books of 7 is  150
Number of products in my pocket 450
total books of 2 is  150
Number of products in my pocket 600
total books of 12 is  150total books of 9 is 
Number of products in my pocket 750
 150
Number of products in my pocket 900
total books of 19 is  150
Number of products in my pocket 1050
total books of 8 is  150
Number of products in my pocket 1200
total books of 5 is  150
Number of products in my pocket 1350
total books of 17

total books of 14 is  150
Number of products in my pocket 15645
total books of 19 is  150
Number of products in my pocket 15795
total books of 18 is  150
Number of products in my pocket 15945
total books of 15 is  150
Number of products in my pocket 16095
total books of 20 is  150
Number of products in my pocket 16245
total books of 21 is  150
Number of products in my pocket 16395
total books of 22 is  150
Number of products in my pocket 16545
total books of 23 is  150
Number of products in my pocket 16695
total books of 27 is  150
Number of products in my pocket 16845
total books of 24 is  150
Number of products in my pocket 16995
total books of 26 is  150
Number of products in my pocket 17145
total books of 29 is  150
Number of products in my pocket 17295
total books of 33 is  150
Number of products in my pocket 17445
total books of 25 is  150
Number of products in my pocket 17595
total books of 34 is  150
Number of products in my pocket 17745
total books of 31 is  150
Number of prod

total books of 33 is  150
Number of products in my pocket 32192
total books of 40 is  149
Number of products in my pocket 32341
total books of 41 is  150
Number of products in my pocket 32491
total books of 42 is  150
Number of products in my pocket 32641
total books of 45 is  150
Number of products in my pocket 32791
total books of 43 is  150
Number of products in my pocket 32941
total books of 55 is  150
Number of products in my pocket 33091
total books of 50 is  150
Number of products in my pocket 33241
total books of 44 is  150
Number of products in my pocket 33391
total books of 46 is  150
Number of products in my pocket 33541
total books of 54 is  150
Number of products in my pocket 33691
total books of 59 is  150
Number of products in my pocket 33841
total books of 47 is  150
Number of products in my pocket 33991
total books of 51 is  150
Number of products in my pocket 34141
total books of 57 is  150
Number of products in my pocket 34291
total books of 53 is  150
Number of prod

total books of 58 is  150
Number of products in my pocket 48487
total books of 64 is  100
Number of products in my pocket 48587
total books of 67 is  100
Number of products in my pocket 48687
total books of 65 is  100
Number of products in my pocket 48787
total books of 66 is  100
Number of products in my pocket 48887
total books of 63 is  100
Number of products in my pocket 48987
total books of 62 is  100
Number of products in my pocket 49087
total books of 71 is  100
Number of products in my pocket 49187
total books of 68 is  100
Number of products in my pocket 49287
total books of 72 is  100
Number of products in my pocket 49387
total books of 74 is  100
Number of products in my pocket 49487
total books of 70 is  100
Number of products in my pocket 49587
total books of 73 is  100
Number of products in my pocket 49687
total books of 69 is  100
Number of products in my pocket 49787
total books of 75 is  100
Number of products in my pocket 49887
total books of 76 is  100
Number of prod

total books of 20 is  149
Number of products in my pocket 64688
total books of 26 is  150
Number of products in my pocket 64838
total books of 22 is  150
Number of products in my pocket 64988
total books of 28 is  150
Number of products in my pocket 65138
total books of 24 is  150
Number of products in my pocket 65288
total books of 31 is  150
Number of products in my pocket 65438
total books of 23 is  150
Number of products in my pocket 65588
total books of 25 is  150
Number of products in my pocket 65738
total books of 33 is  150
Number of products in my pocket 65888
total books of 35 is  150
Number of products in my pocket 66038
total books of 27 is  150
Number of products in my pocket 66188
total books of 37 is  150
Number of products in my pocket 66338
total books of 34 is  150
Number of products in my pocket 66488
total books of 36 is  150
Number of products in my pocket 66638
total books of 32 is  150
Number of products in my pocket 66788
total books of 38 is  150
Number of prod

total books of 13 is  150
Number of products in my pocket 82777
total books of 20 is  88
Number of products in my pocket 82865
total books of 16 is  total books of 17 is  150
Number of products in my pocket 83015
150total books of 14 is  150
Number of products in my pocket 83165

Number of products in my pocket 83315
total books of 18 is  150
Number of products in my pocket 83465
total books of 7 is  150
Number of products in my pocket 83615
total books of 11 is  150
Number of products in my pocket 83765
total books of 12 is  150
Number of products in my pocket 83915
total books of 9 is  150
Number of products in my pocket 84065
total books of 15 is  150
Number of products in my pocket 84215
total books of 10 is  150
Number of products in my pocket 84365
885 y-hoc
total pages 13
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
total books of 0 is  150
Number of products in my pocket 84515
total books of 1 is  150
Number of products in my pocket 84665
total books of 13 is  29
Number of pr

In [84]:
print(f"Number of products in my pocket {booksQueue.qsize()}")

Number of products in my pocket 97102


In [84]:
df = pd.DataFrame(a, columns=['Tên Sách', 'Tên tác giả','Thể loại','Nội dung tóm tắt','Giá bìa'],index=[0])
while not booksQueue.empty():
    df = df.append(booksQueue.get(),ignore_index=True)

NameError: name 'a' is not defined

In [86]:
df.to_csv("tiki_book.csv",)